In [15]:
csv_file_path = "./content/output4h.csv"

In [16]:
!git clone https://github.com/ztuntrade/untrade-sdk.git
!pip install ./untrade-sdk/.

import uuid
import os
from untrade.client import Client

fatal: destination path 'untrade-sdk' already exists and is not an empty directory.


Processing c:\users\haris\desktop\zelta\untrade-sdk
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5100 sha256=0a784c9993bca0ac451fd772781cbddeefed2f12d4d0a941edecc7aa6c166c83
  Stored in directory: c:\users\haris\appdata\local\pip\cache\wheels\0c\d6\bc\ca59932606376648b7a1f7cc02d1e8d67f1c16576d13bb06bb
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      Successfully uninstalled untrade-0.1.0



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
def perform_backtest(csv_file_path):
     client = Client()
     result = client.backtest(
         jupyter_id="test",
         file_path=csv_file_path,
         leverage=1,
     )
     return result


def perform_backtest_large_csv(csv_file_path):
    client = Client()
    file_id = str(uuid.uuid4())
    chunk_size = 90 * 1024 * 1024  # 90 MB
    total_size = os.path.getsize(csv_file_path)
    total_chunks = (total_size + chunk_size - 1) // chunk_size
    chunk_number = 0

    # Handle small files
    if total_size <= chunk_size:
        total_chunks = 1
        result = client.backtest(
            file_path=csv_file_path,
            leverage=1,
            jupyter_id="test",
        )
        for value in result:
            print(value)
        return result

    # Process large files in chunks
    with open(csv_file_path, "rb") as f:
        while True:
            chunk_data = f.read(chunk_size)
            if not chunk_data:
                break

            # Save each chunk temporarily in /tmp
            chunk_file_path = f"/tmp/{file_id}_chunk{chunk_number}.csv"
            with open(chunk_file_path, "wb") as chunk_file:
                chunk_file.write(chunk_data)

            # Perform backtest on the current chunk
            result = client.backtest(
                file_path=chunk_file_path,
                leverage=1,
                jupyter_id="test",
                file_id=file_id,
                chunk_number=chunk_number,
                total_chunks=total_chunks,
            )

            # Process the results of the backtest
            for value in result:
                print(value)

            # Remove the temporary chunk file
            os.remove(chunk_file_path)

            # Move to the next chunk
            chunk_number += 1

    return result

In [18]:
#change to perform_backtest_large_csv(csv_file_path) for large files
backtest_result = perform_backtest(csv_file_path)
print(backtest_result)
for value in backtest_result:
    print(value)

<generator object Client._handle_response_stream at 0x000001DEC6CFC2B0>
data: {
  "jupyter_id": "test",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 16:00:00",
      "Total Trades": 34,
      "Leverage Applied": 1.0,
      "Winning Trades": 15,
      "Losing Trades": 19,
      "No. of Long Trades": 17,
      "No. of Short Trades": 17,
      "Benchmark Return(%)": 309.761202,
      "Benchmark Return(on $1000)": 3097.61202,
      "Win Rate": 44.117647,
      "Winning Streak": 4,
      "Losing Streak": 5,
      "Gross Profit": 2311.922742,
      "Net Profit": 2260.922742,
      "Average Profit": 66.497728,
      "Maximum Drawdown(%)": 18.479636,
      "Average Drawdown(%)": 7.183761,
      "Largest Win": 1751.617421,
      "Average Win": 265.311325,
      "Largest Loss": -237.543864,
      "Average Loss": -90.460375,
      "Maximum Holding Time": "103 days 12:0:0",
      "Average Holding Time": "37 days 4:21